In [17]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile 
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range 


In [18]:
pickle_file='notMNIST.pickle'


with open(pickle_file , 'rb') as f:
    save=pickle.load(f)
    train_dataset=save ['train_dataset']
    train_labels=save ['train_labels']
    valid_dataset=save ['valid_dataset']
    valid_labels=save ['valid_labels']
    test_dataset=save ['test_dataset']
    test_labels=save ['test_labels']
    del save #to help compiler to free up space 
    print('train_dataset', train_dataset.shape , train_labels.shape)
    print('validation set ', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_labels.shape)

train_dataset (200000, 28, 28) (200000,)
validation set  (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [19]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [20]:
def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)== np.argmax(labels, 1))/ predictions.shape[0])

In [21]:
batch_size=128
num_nodes=1024
#SGDSTEP=0.5
#NumberofLayers=2 #(front layer+1 hidden layer)

'''
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial:tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
        '''
    
graph=tf.Graph()
with graph.as_default():
    # Input data.For the training data,we use a placeholder to fed with a training mini batch
    #at runtime with a training min i batch
    tf_train_dataset= tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1=tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
    biases_1=tf.Variable(tf.zeros([num_nodes]))
    weights_2=tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2=tf.Variable(tf.zeros(num_labels))
    
    #training Computation.
    relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+ biases_1)
    logits=tf.matmul(relu_layer,weights_2)+biases_2
    #loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions for training
    train_prediction=tf.nn.softmax(logits)
    valid_prediction=tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+biases_1),weights_2)+biases_2)
    test_predicton= tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1),weights_2)+biases_2)
    
    
   
    print(graph)

In [22]:
num_steps=30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    c=0
    step_array=[]
    accuracy_array=[]
    
    for step in range(num_steps):
        #Pick an offset within taining data. which has been randomized.
        # Note: we could use better randomization across epochs
        offset= (step*batch_size) % (train_labels.shape[0] - batch_size) # we have used train_labels.shape[0] it is equal to the toatal no of image in the datasets
        
        #generate a minibatch , as SGD requires them
        batch_data= train_dataset[offset:(offset+batch_size),:]
        batch_labels= train_labels[offset:(offset+batch_size),:]
        
        #Prepare a dictionary telling the session where to feed the minibatch
        # The Key of the dictinary is the placeholder node of the graph to be fed,
        #and the value is the numpy array to feed to it.
        feed_dict={tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _, l,predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500==0):
            miniAccuracy=accuracy( valid_prediction.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step,l))
            print("Minibatch accurracy: %.1f%%" % miniAccuracy)
            print (c)
            step_array.append(step)
            accuracy_array.append(miniAccuracy)
            c=c+1
            plt.plot(step,miniAccuracy)
        print("Test Accuracy: %.1f%%" % accuracy(test_predicton.eval(),test_labels))
session.close()

Initialized
Minibatch loss at step 0: 369.530823
Minibatch accurracy: 35.1%
0
Test Accuracy: 37.3%
Test Accuracy: 39.5%
Test Accuracy: 29.6%
Test Accuracy: 52.5%
Test Accuracy: 54.0%
Test Accuracy: 64.3%
Test Accuracy: 67.2%
Test Accuracy: 76.5%
Test Accuracy: 70.0%
Test Accuracy: 81.5%
Test Accuracy: 82.1%
Test Accuracy: 83.9%
Test Accuracy: 83.9%
Test Accuracy: 84.5%
Test Accuracy: 79.6%
Test Accuracy: 77.2%
Test Accuracy: 81.8%
Test Accuracy: 83.4%
Test Accuracy: 81.7%
Test Accuracy: 81.4%
Test Accuracy: 82.4%
Test Accuracy: 84.9%
Test Accuracy: 82.5%
Test Accuracy: 84.5%
Test Accuracy: 84.4%
Test Accuracy: 84.7%
Test Accuracy: 85.3%
Test Accuracy: 79.7%
Test Accuracy: 81.2%
Test Accuracy: 81.7%
Test Accuracy: 83.9%
Test Accuracy: 78.9%
Test Accuracy: 82.1%
Test Accuracy: 84.2%
Test Accuracy: 84.8%
Test Accuracy: 80.7%
Test Accuracy: 85.6%
Test Accuracy: 85.5%
Test Accuracy: 85.8%
Test Accuracy: 84.4%
Test Accuracy: 85.4%


KeyboardInterrupt: 